In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
trainning_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

100%|██████████| 26421880/26421880 [00:08<00:00, 3166110.33it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 153236.33it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:03<00:00, 1108630.18it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 17756806.74it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

train_dataloader = DataLoader(trainning_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()##???? transforms to a 1d tensor I believe
        self.linear_relu_stack = nn.Sequential(##????
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()#????
    for batch, (X,y) in enumerate(dataloader):#for every batch
        X, y = X.to(device), y.to(device)#load the batch into the relevant ram

        pred = model(X)#make the prediction
        loss = loss_fn(pred, y)#calculate the loss

        loss.backward()#generate the gradients I believe
        optimizer.step()#update the weights
        optimizer.zero_grad()#???

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X) 
            print(f"loss: {loss:>7f} [{current:>5d}]/{size:>5d}")



In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():#??
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()#not fully sure on this
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.308897 [   64]/60000
loss: 2.297882 [ 6464]/60000
loss: 2.282094 [12864]/60000
loss: 2.271558 [19264]/60000
loss: 2.239869 [25664]/60000
loss: 2.226418 [32064]/60000
loss: 2.228252 [38464]/60000
loss: 2.202623 [44864]/60000
loss: 2.199702 [51264]/60000
loss: 2.158509 [57664]/60000
Test Error: 
 Accuracy: 38.0%, Avg loss: 2.158586 

Epoch 2
----------------------------
loss: 2.178202 [   64]/60000
loss: 2.163158 [ 6464]/60000
loss: 2.112112 [12864]/60000
loss: 2.123219 [19264]/60000
loss: 2.049690 [25664]/60000
loss: 2.009838 [32064]/60000
loss: 2.029499 [38464]/60000
loss: 1.958926 [44864]/60000
loss: 1.968474 [51264]/60000
loss: 1.887875 [57664]/60000
Test Error: 
 Accuracy: 47.2%, Avg loss: 1.891197 

Epoch 3
----------------------------
loss: 1.931563 [   64]/60000
loss: 1.893898 [ 6464]/60000
loss: 1.789779 [12864]/60000
loss: 1.828081 [19264]/60000
loss: 1.690923 [25664]/60000
loss: 1.665911 [32064]/60000
loss: 1.676944 [38464]/60000
lo